Create a new Python application

In [1]:
pip install azure-cognitiveservices-vision-customvision

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Azure의 Custom Vision 라이브러리를 추가. 예측을 위하여 prediction을 포함
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
# OpenAPI 스펙에 맞춰서 Authentication을 처리할 수 있도록 해주는 코드
from msrest.authentication import ApiKeyCredentials
# Matplotlib의 pyplot을 포함하여 예측 결과를 그리기
from matplotlib import pyplot as plt
# Python Image 라이브러리로 이미지 그리기
from PIL import Image, ImageDraw, ImageFont
# Python Numpy (수학 및 과학 연산 패키지) 포함
import numpy as np
# 파일 처리 작업을 위해 os 라이브러리 포함
import os

In [28]:
# 사용자가 만든 AI 모델의 예측 기능을 사용하기 위한 endpoint 지정
prediction_endpoint = "https://southcentralus.api.cognitive.microsoft.com" #이거 수정하세요
# KEY 값 지정
prediction_key = "9d4f2bbf08f544b4a700983e66874ee4" #이거 수정하세요
# 프로젝트 ID 지정
project_id = "02a5329b-ccdf-4f91-b0b3-7345baca5c4e" #이거 수정하세요
# 모델명 지정
model_name = "Iteration8" #이거 수정하세요

In [3]:
# 앞에서 지정한 API KEY를 써서 커스텀 비전 모델을 사용할 클라이언트를 인증
credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
# endpoint를 써서 클라이언트 등록
predictor = CustomVisionPredictionClient(endpoint=prediction_endpoint, credentials=credentials)

NameError: name 'prediction_key' is not defined

In [ ]:
import os
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

# 테스트 파일이 저장된 디렉토리 경로와 출력 파일 경로
input_dir = "D:/msaischool/test/input/kickscooter-test-images2" #이거 수정하세요
output_dir = "D:/msaischool/test/output/kickscooter-20241105" #이거 수정하세요

# 출력 디렉토리가 없으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 디렉토리 내 모든 'test-'로 시작하는 파일을 순회
for image_file in os.listdir(input_dir):
    if image_file.endswith(".jpg"):
        full_image_path = os.path.join(input_dir, image_file)
        print(f'Detecting objects in {full_image_path}')

        # Python Imaging Library의 image open함수를 써서 테스트 이미지 파일 오픈
        image = Image.open(full_image_path)

        # Numpy에서 이미지의 shape을 높이, 폭, 채널 읽기
        h, w, ch = np.array(image).shape
        print(h, w, ch)

        # 테스트 이미지를 열고 모델에 적용해서 결과를 저장
        with open(full_image_path, mode="rb") as image_data:
            results = predictor.detect_image(project_id, model_name, image_data)

        # 그래프 크기 지정하고 축 비활성화
        fig = plt.figure(figsize=(8, 8))
        plt.axis('off')

        # 개체 인식 박스를 magenta로 지정
        draw = ImageDraw.Draw(image)
        lineWidth = int(w / 100)
        color = 'magenta'

        # 예측된 객체들에 대해 확률이 50% 이상인 경우 바운딩 박스 그리기
        for prediction in results.predictions:
            if prediction.probability * 100 > 50:
                # 바운딩 박스 좌표 계산
                left = prediction.bounding_box.left * w
                top = prediction.bounding_box.top * h
                width = prediction.bounding_box.width * w
                height = prediction.bounding_box.height * h

                # 바운딩 박스를 그리기 위한 좌표 설정
                points = [(left, top), (left + width, top), (left + width, top + height), (left, top + height), (left, top)]

                # 바운딩 박스를 magenta 색으로 그림
                draw.line(points, fill=color, width=lineWidth)

                # 바운딩 박스에 태그 이름과 확률 표시
                plt.annotate(f'{prediction.tag_name} {prediction.probability * 100:.2f}%', (left, top), color=color)

        # 이미지를 Numpy 배열로 변환하여 Matplotlib에 표시
        plt.imshow(np.array(image))

        # 파일 이름에 맞춰 output 폴더에 저장
        output_file_path = os.path.join(output_dir, image_file)
        fig.savefig(output_file_path)
        print(f'Results saved in {output_file_path}')



In [30]:
import os
import xml.etree.ElementTree as ET
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, ImageFileCreateBatch, Region
from msrest.authentication import ApiKeyCredentials
from PIL import Image

# Azure Custom Vision 설정
ENDPOINT = "https://mycustomvision-prediction.cognitiveservices.azure.com"  # Custom Vision의 엔드포인트
TRAINING_KEY = "55qgYtEP1itSIa15LGdJN6YWVjVCJS0rYFFr2R2sEuBMTfYvIrOjJQQJ99AJACL93NaXJ3w3AAAIACOGcmyK"  # Custom Vision의 Training Key
PROJECT_ID = "cf9ed873-e58b-4cbf-9295-c2ba83fd0b68"  # Custom Vision 프로젝트 ID

credentials = ApiKeyCredentials(in_headers={"Training-key": TRAINING_KEY})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Custom Vision 프로젝트에서 'Kick-scooter' 태그 가져오기
tags = trainer.get_tags(PROJECT_ID)
kick_scooter_tag = next((t for t in tags if t.name == 'Kick-scooter'), None)

if not kick_scooter_tag:
    raise ValueError("프로젝트에 'Kick-scooter' 태그가 존재하지 않습니다. Custom Vision 프로젝트에서 태그를 생성하세요.")

# Pascal VOC XML 라벨 경로 및 이미지 경로 지정
base_path = 'D:/msaischool/test/kickboard.v6i.voc/train'

# XML 파일 순회
image_entries = []

for label_file in os.listdir(base_path):
    if label_file.endswith('.xml'):
        tree = ET.parse(os.path.join(base_path, label_file))
        root = tree.getroot()
        
        # 이미지 파일명 가져오기
        image_filename = root.find('filename').text
        image_filepath = os.path.join(base_path, image_filename)
        
        # 이미지가 존재하는지 확인
        if os.path.exists(image_filepath):
            with Image.open(image_filepath) as img:
                img_width, img_height = img.size
            
            # 바운딩 박스 정보 추출
            regions = []
            for obj in root.findall('object'):
                tag_name = obj.find('name').text
                if tag_name.lower() == 'kickboard':  # 'kickboard'를 'Kick-scooter'로 변환
                    bndbox = obj.find('bndbox')
                    x_min = int(bndbox.find('xmin').text)
                    y_min = int(bndbox.find('ymin').text)
                    x_max = int(bndbox.find('xmax').text)
                    y_max = int(bndbox.find('ymax').text)
                    
                    # 바운딩 박스 조정
                    if x_max > img_width:
                        x_max = img_width
                    if y_max > img_height:
                        y_max = img_height
                    
                    # 유효성 검사
                    if x_min >= 0 and y_min >= 0 and x_max > x_min and y_max > y_min:
                        regions.append(Region(
                            tag_id=kick_scooter_tag.id,
                            left=x_min / img_width,
                            top=y_min / img_height,
                            width=(x_max - x_min) / img_width,
                            height=(y_max - y_min) / img_height
                        ))
                    else:
                        print(f"이미지 {image_filename}에 잘못된 바운딩 박스 발견: ({x_min}, {y_min}, {x_max}, {y_max})")

            # 이미지 엔트리 생성
            if regions:  # 유효한 바운딩 박스가 있는 경우에만 업로드
                with open(image_filepath, "rb") as image_contents:
                    image_data = image_contents.read()
                    image_entry = ImageFileCreateEntry(
                        name=image_filename,
                        contents=image_data,
                        regions=regions
                    )
                    image_entries.append(image_entry)

# Custom Vision에 이미지 업로드 (64개씩 나누어 처리)
batch_size = 64
for i in range(0, len(image_entries), batch_size):
    batch = ImageFileCreateBatch(images=image_entries[i:i + batch_size])
    upload_result = trainer.create_images_from_files(PROJECT_ID, batch)
    
    # 업로드 결과 확인
    if upload_result.is_batch_successful:
        print(f"배치 {i // batch_size + 1} 업로드 성공")
    else:
        print(f"배치 {i // batch_size + 1} 업로드 실패")
        for image in upload_result.images:
            if not image.status == 'OK':
                print(f"이미지 {image.source_url} 업로드 실패, 상태: {image.status}")

print("모든 이미지 업로드 및 태그 추가 완료")


배치 1 업로드 실패
이미지 IMG_3584_mov-0009_jpg.rf.b6c087e56688ac2f4da36b8cfa4fe2c6.jpg 업로드 실패, 상태: OKDuplicate
이미지 IMG_3584_mov-0019_jpg.rf.3bdcef39158d54c1a6b3c819ec03fc7b.jpg 업로드 실패, 상태: OKDuplicate
이미지 IMG_3584_mov-0004_jpg.rf.dce66301cad479223bc85bb7f947eb01.jpg 업로드 실패, 상태: OKDuplicate
배치 2 업로드 실패
이미지 IMG_3584_mov-0052_jpg.rf.fc705899a892e49cd6cdb07d9b2b630e.jpg 업로드 실패, 상태: OKDuplicate
이미지 IMG_3584_mov-0058_jpg.rf.f9a9584be3e041d02c597a9294f5d9b4.jpg 업로드 실패, 상태: OKDuplicate
이미지 IMG_3584_mov-0040_jpg.rf.9c62a9cee04e8bc6ac552dbd75f0deb2.jpg 업로드 실패, 상태: OKDuplicate
배치 3 업로드 성공
배치 4 업로드 성공
배치 5 업로드 실패
이미지 IMG_3585_mov-0133_jpg.rf.d76f67f20394fbb0c06bea0e2ced1831.jpg 업로드 실패, 상태: OKDuplicate
이미지 IMG_3585_mov-0126_jpg.rf.ee7ccdc102c4b8329cb640112419dc9d.jpg 업로드 실패, 상태: OKDuplicate
배치 6 업로드 실패
이미지 IMG_3585_mov-0135_jpg.rf.f27ad5c7480f55a1aadb3c4d90c26534.jpg 업로드 실패, 상태: OKDuplicate
배치 7 업로드 실패
이미지 IMG_3585_mov-0194_jpg.rf.e7850ad13eefcaece963c90204c9b748.jpg 업로드 실패, 상태: OKDuplicate
배치 8 업로드 실패
이미지 

In [8]:
import os
import xml.etree.ElementTree as ET
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, ImageFileCreateBatch, Region
from msrest.authentication import ApiKeyCredentials
from PIL import Image

# Azure Custom Vision 설정
ENDPOINT = "https://cv5b015.cognitiveservices.azure.com/"  # Custom Vision의 엔드포인트
TRAINING_KEY = "28jIEsv0cxpoFWKq41DwjHSBMcMh2VQsG7II5yt6h7dbFk7sW7siJQQJ99AJACL93NaXJ3w3AAAJACOGdgJG"  # Custom Vision의 Training Key
PROJECT_ID = "cf9ed873-e58b-4cbf-9295-c2ba83fd0b68"  # Custom Vision 프로젝트 ID

credentials = ApiKeyCredentials(in_headers={"Training-key": TRAINING_KEY})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Custom Vision 프로젝트에서 'Unhelmet' 태그 가져오기
tags = trainer.get_tags(PROJECT_ID)
unhelmet_tag = next((t for t in tags if t.name == 'unhelmet'), None)

if not unhelmet_tag:
    raise ValueError("프로젝트에 'unhelmet' 태그가 존재하지 않습니다. Custom Vision 프로젝트에서 태그를 생성하세요.")

# Pascal VOC XML 라벨 경로 및 이미지 경로 지정
base_path = 'D:/msaischool/9team_kickscooter/20241106/Head Direction.v1i.voc/train'

# XML 파일 순회
image_entries = []

for label_file in os.listdir(base_path):
    if label_file.endswith('.xml'):
        tree = ET.parse(os.path.join(base_path, label_file))
        root = tree.getroot()
        
        # 이미지 파일명 가져오기
        image_filename = root.find('filename').text
        image_filepath = os.path.join(base_path, image_filename)
        
        # 이미지가 존재하는지 확인
        if os.path.exists(image_filepath):
            with Image.open(image_filepath) as img:
                img_width, img_height = img.size
            
            # 바운딩 박스 정보 추출 ('0' 태그만 포함)
            regions = []
            for obj in root.findall('object'):
                tag_name = obj.find('name').text
                if tag_name == '0':  # '0'을 'Unhelmet' 태그로 간주
                    bndbox = obj.find('bndbox')
                    x_min = int(bndbox.find('xmin').text)
                    y_min = int(bndbox.find('ymin').text)
                    x_max = int(bndbox.find('xmax').text)
                    y_max = int(bndbox.find('ymax').text)
                    
                    # 바운딩 박스 조정
                    if x_max > img_width:
                        x_max = img_width
                    if y_max > img_height:
                        y_max = img_height
                    
                    # 유효성 검사
                    if x_min >= 0 and y_min >= 0 and x_max > x_min and y_max > y_min:
                        regions.append(Region(
                            tag_id=unhelmet_tag.id,
                            left=x_min / img_width,
                            top=y_min / img_height,
                            width=(x_max - x_min) / img_width,
                            height=(y_max - y_min) / img_height
                        ))
                    else:
                        print(f"이미지 {image_filename}에 잘못된 바운딩 박스 발견: ({x_min}, {y_min}, {x_max}, {y_max})")

            # 이미지 엔트리 생성
            if regions:  # 유효한 바운딩 박스가 있는 경우에만 업로드
                with open(image_filepath, "rb") as image_contents:
                    image_data = image_contents.read()
                    image_entry = ImageFileCreateEntry(
                        name=image_filename,
                        contents=image_data,
                        regions=regions
                    )
                    image_entries.append(image_entry)

# Custom Vision에 이미지 업로드 (64개씩 나누어 처리)
batch_size = 64
for i in range(0, len(image_entries), batch_size):
    batch = ImageFileCreateBatch(images=image_entries[i:i + batch_size])
    upload_result = trainer.create_images_from_files(PROJECT_ID, batch)
    
    # 업로드 결과 확인
    if upload_result.is_batch_successful:
        print(f"배치 {i // batch_size + 1} 업로드 성공")
    else:
        print(f"배치 {i // batch_size + 1} 업로드 실패")
        for image in upload_result.images:
            if not image.status == 'OK':
                print(f"이미지 {image.source_url} 업로드 실패, 상태: {image.status}")

print("모든 unhelmet 이미지 업로드 및 태그 추가 완료")


배치 1 업로드 성공
배치 2 업로드 성공
배치 3 업로드 성공
배치 4 업로드 성공
배치 5 업로드 성공
배치 6 업로드 성공
모든 unhelmet 이미지 업로드 및 태그 추가 완료


In [ ]:
import os
import xml.etree.ElementTree as ET
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, ImageFileCreateBatch, Region
from msrest.authentication import ApiKeyCredentials
from PIL import Image

# Azure Custom Vision 설정
ENDPOINT = "https://mycustomvision-prediction.cognitiveservices.azure.com"  # Custom Vision의 엔드포인트
TRAINING_KEY = "55qgYtEP1itSIa15LGdJN6YWVjVCJS0rYFFr2R2sEuBMTfYvIrOjJQQJ99AJACL93NaXJ3w3AAAIACOGcmyK"  # Custom Vision의 Training Key
PROJECT_ID = "cf9ed873-e58b-4cbf-9295-c2ba83fd0b68"  # Custom Vision 프로젝트 ID

credentials = ApiKeyCredentials(in_headers={"Training-key": TRAINING_KEY})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Custom Vision 프로젝트에서 'Helmet' 태그 가져오기
tags = trainer.get_tags(PROJECT_ID)
helmet_tag = next((t for t in tags if t.name == 'Helmet'), None)

if not helmet_tag:
    raise ValueError("프로젝트에 'Helmet' 태그가 존재하지 않습니다. Custom Vision 프로젝트에서 태그를 생성하세요.")

# Pascal VOC XML 라벨 경로 및 이미지 경로 지정
base_path = 'D:/msaischool/test/kickboard.v6i.voc/train'

# XML 파일 순회
image_entries = []

for label_file in os.listdir(base_path):
    if label_file.endswith('.xml'):
        tree = ET.parse(os.path.join(base_path, label_file))
        root = tree.getroot()
        
        # 이미지 파일명 가져오기
        image_filename = root.find('filename').text
        image_filepath = os.path.join(base_path, image_filename)
        
        # 이미지가 존재하는지 확인
        if os.path.exists(image_filepath):
            with Image.open(image_filepath) as img:
                img_width, img_height = img.size
            
            # 바운딩 박스 정보 추출 (Helmet만 포함)
            regions = []
            for obj in root.findall('object'):
                tag_name = obj.find('name').text
                if tag_name.lower() == 'helmet':  # 'helmet' 태그만 포함
                    bndbox = obj.find('bndbox')
                    x_min = int(bndbox.find('xmin').text)
                    y_min = int(bndbox.find('ymin').text)
                    x_max = int(bndbox.find('xmax').text)
                    y_max = int(bndbox.find('ymax').text)
                    
                    # 바운딩 박스 조정
                    if x_max > img_width:
                        x_max = img_width
                    if y_max > img_height:
                        y_max = img_height
                    
                    # 유효성 검사
                    if x_min >= 0 and y_min >= 0 and x_max > x_min and y_max > y_min:
                        regions.append(Region(
                            tag_id=helmet_tag.id,
                            left=x_min / img_width,
                            top=y_min / img_height,
                            width=(x_max - x_min) / img_width,
                            height=(y_max - y_min) / img_height
                        ))
                    else:
                        print(f"이미지 {image_filename}에 잘못된 바운딩 박스 발견: ({x_min}, {y_min}, {x_max}, {y_max})")

            # 이미지 엔트리 생성
            if regions:  # 유효한 바운딩 박스가 있는 경우에만 업로드
                with open(image_filepath, "rb") as image_contents:
                    image_data = image_contents.read()
                    image_entry = ImageFileCreateEntry(
                        name=image_filename,
                        contents=image_data,
                        regions=regions
                    )
                    image_entries.append(image_entry)

# Custom Vision에 이미지 업로드 (64개씩 나누어 처리)
batch_size = 64
for i in range(0, len(image_entries), batch_size):
    batch = ImageFileCreateBatch(images=image_entries[i:i + batch_size])
    upload_result = trainer.create_images_from_files(PROJECT_ID, batch)
    
    # 업로드 결과 확인
    if upload_result.is_batch_successful:
        print(f"배치 {i // batch_size + 1} 업로드 성공")
    else:
        print(f"배치 {i // batch_size + 1} 업로드 실패")
        for image in upload_result.images:
            if not image.status == 'OK':
                print(f"이미지 {image.source_url} 업로드 실패, 상태: {image.status}")

print("모든 헬멧 이미지 업로드 및 태그 추가 완료")

In [31]:
import os
import cv2
import numpy as np
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from PIL import Image, ImageDraw

# Azure Custom Vision 설정 - 킥보드
prediction_endpoint1 = "https://southcentralus.api.cognitive.microsoft.com"
prediction_key1 = "9d4f2bbf08f544b4a700983e66874ee4"
project_id_model1 = "02a5329b-ccdf-4f91-b0b3-7345baca5c4e"
model_name_model1 = "Iteration5"

# Azure Custom Vision 설정 - 헬멧
prediction_endpoint2 = "https://mycustomvision-prediction.cognitiveservices.azure.com"
prediction_key2 = "55qgYtEP1itSIa15LGdJN6YWVjVCJS0rYFFr2R2sEuBMTfYvIrOjJQQJ99AJACL93NaXJ3w3AAAIACOGcmyK"
project_id_model2 = "cf9ed873-e58b-4cbf-9295-c2ba83fd0b68"
model_name_model2 = "Iteration7"

# 예측 클라이언트 생성
credentials1 = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key1})
predictor_model1 = CustomVisionPredictionClient(endpoint=prediction_endpoint1, credentials=credentials1)

credentials2 = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key2})
predictor_model2 = CustomVisionPredictionClient(endpoint=prediction_endpoint2, credentials=credentials2)

# 비디오에서 프레임 저장
def save_frames(video_path, output_dir, frame_interval):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return []

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    frame_count = 0
    # kick_scooter_images = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 지정된 간격에 따라 프레임 저장
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_dir, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_filename, frame)
            print(f"Saved frame: {frame_filename}")

        frame_count += 1

    cap.release()

# 저장된 프레임 이미지에서 객체 인식
def detect_objects_kick_image(image_path):
    print("Detecting objects in a frame")

    # 이미지 열기
    image = Image.open(image_path)
    h, w, ch = np.array(image).shape
    print(f"Image size: {h}x{w} with {ch} channels")

    # 첫 번째 모델 예측
    with open(image_path, mode="rb") as image_data: 
        results_model1 = predictor_model1.detect_image(project_id_model1, model_name_model1, image_data)
    
    draw = ImageDraw.Draw(image)
    lineWidth = int(w / 200)

    # 첫 번째 모델 결과: 파란색
    kick_scooter_detected = False  # Kick-scooter 감지 여부

    for prediction in results_model1.predictions:
        if prediction.tag_name == "Kick-scooter" and prediction.probability * 100 > 10:
            kick_scooter_detected = True  # Kick-scooter 감지됨
            left = prediction.bounding_box.left * w
            top = prediction.bounding_box.top * h
            width = prediction.bounding_box.width * w
            height = prediction.bounding_box.height * h
            
            # 바운딩 박스 그리기
            draw.rectangle([left, top, left + width, top + height], outline='blue', width=lineWidth)
            draw.text((left, top), f'{prediction.tag_name} {prediction.probability * 100:.2f}%', fill='blue')

    # 결과 이미지 저장 (Kick-scooter가 감지된 경우에만)
    if kick_scooter_detected:
        output_file_path = os.path.join(kick_scooter_dir, os.path.basename(image_path))
        image.save(output_file_path)
        print(f'Results saved in {output_file_path}')
        return True  # Kick-scooter가 감지된 경우 True 반환
    return False  # 감지되지 않은 경우 False 반환


def detect_objects_helmet_image(image_path):
    print("Detecting objects in a kick-scooter")

    # 이미지 열기
    image = Image.open(image_path)
    h, w, ch = np.array(image).shape
    print(f"Image size: {h}x{w} with {ch} channels")

    # 첫 번째 모델 예측
    with open(image_path, mode="rb") as image_data: 
        results_model2 = predictor_model2.detect_image(project_id_model2, model_name_model2, image_data)
    
    draw = ImageDraw.Draw(image)
    lineWidth = int(w / 200)

    # 두 번째 모델 결과: 마젠타
    for prediction in results_model2.predictions:
        if prediction.tag_name == "unhelmet" and prediction.probability * 100 > 80:
            left = prediction.bounding_box.left * w
            top = prediction.bounding_box.top * h
            width = prediction.bounding_box.width * w
            height = prediction.bounding_box.height * h

            draw.rectangle([left, top, left + width, top + height], outline='magenta', width=lineWidth)
            draw.text((left, top), f'{prediction.tag_name} {prediction.probability * 100:.2f}%', fill='magenta')

    output_file_path = os.path.join(helmet_dir, os.path.basename(image_path))
    image.save(output_file_path)
    print(f'Results saved in {output_file_path}')

if __name__ == "__main__":
    video_path = "D:/msaischool/videotest/test.mp4"  
    output_dir = "D:/msaischool/videotest/output_frames"  
    kick_scooter_dir = "D:/msaischool/videotest/kick_scooter_frames"
    helmet_dir = "D:/msaischool/videotest/helmet_out"  

    # 디렉토리 생성
    for directory in [output_dir, kick_scooter_dir, helmet_dir]:
        if not os.path.exists(directory):
            os.makedirs(directory)

    frame_interval = 2

    save_frames(video_path, output_dir, frame_interval)

    # 첫 번째 모델 처리
    for image_file in os.listdir(output_dir):
        if image_file.endswith(".jpg"):
            kick_image_path = os.path.join(output_dir, image_file)
            if detect_objects_kick_image(kick_image_path):  # Kick-scooter가 감지된 경우에만
                pass  # 이미지를 저장했으므로 여기서는 아무 작업도 하지 않음

    # 두 번째 모델 처리
    for image_file in os.listdir(kick_scooter_dir):
        if image_file.endswith(".jpg"):
            helmet_image_path = os.path.join(kick_scooter_dir, image_file)
            detect_objects_helmet_image(helmet_image_path)  # 두 번째 모델 처리

Saved frame: D:/msaischool/videotest/output_frames\frame_0.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_2.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_4.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_6.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_8.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_10.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_12.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_14.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_16.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_18.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_20.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_22.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_24.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_26.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_28.jpg
Saved frame: D:/msaischool/videotest/output_f

KeyboardInterrupt: 

In [34]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
import os
import cv2
import asyncio
import aiohttp
import shutil
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from PIL import Image, ImageDraw
from io import BytesIO
import re

# Azure Custom Vision 설정 - 킥보드
prediction_endpoint1 = "https://southcentralus.api.cognitive.microsoft.com"
prediction_key1 = "9d4f2bbf08f544b4a700983e66874ee4"
project_id_model1 = "02a5329b-ccdf-4f91-b0b3-7345baca5c4e"
model_name_model1 = "Iteration5"

# Azure Custom Vision 설정 - 헬멧
prediction_endpoint2 = "https://mycustomvision-prediction.cognitiveservices.azure.com"
prediction_key2 = "55qgYtEP1itSIa15LGdJN6YWVjVCJS0rYFFr2R2sEuBMTfYvIrOjJQQJ99AJACL93NaXJ3w3AAAIACOGcmyK"
project_id_model2 = "cf9ed873-e58b-4cbf-9295-c2ba83fd0b68"
model_name_model2 = "Iteration7"

# 파일명에 포함된 숫자를 기준으로 자연스러운 정렬을 위한 함수
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]

# 예측 클라이언트 생성
credentials1 = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key1})
predictor_model1 = CustomVisionPredictionClient(endpoint=prediction_endpoint1, credentials=credentials1)

credentials2 = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key2})
predictor_model2 = CustomVisionPredictionClient(endpoint=prediction_endpoint2, credentials=credentials2)

# 비동기 예측 함수
async def async_detect_objects(image_path, predictor_endpoint, api_key, project_id, model_name, tag_name, output_dir, color, probability_threshold):
    async with aiohttp.ClientSession() as session:
        image = Image.open(image_path)
        w, h = image.size
        draw = ImageDraw.Draw(image)

        with open(image_path, "rb") as image_data:
            payload = image_data.read()
            headers = {
                "Prediction-key": api_key,
                "Content-Type": "application/octet-stream"
            }
            async with session.post(
                f"{predictor_endpoint}/customvision/v3.0/Prediction/{project_id}/detect/iterations/{model_name}/image",
                data=payload,
                headers=headers
            ) as response:
                response_json = await response.json()

        detected = False
        for prediction in response_json.get("predictions", []):
            if prediction["tagName"] == tag_name and prediction["probability"] * 100 > probability_threshold:
                detected = True
                left = prediction["boundingBox"]["left"] * w
                top = prediction["boundingBox"]["top"] * h
                width = prediction["boundingBox"]["width"] * w
                height = prediction["boundingBox"]["height"] * h
                draw.rectangle([left, top, left + width, top + height], outline=color, width=2)
                draw.text((left, top), f'{prediction["tagName"]} {prediction["probability"] * 100:.2f}%', fill=color)

        # 바운딩박스가 있든 없든 이미지 저장
        output_file_path = os.path.join(output_dir, os.path.basename(image_path))
        image.save(output_file_path)
        print(f'Results saved in {output_file_path}')

# 비동기 이미지 처리 루프
async def process_images_async(image_dir, predictor_endpoint, api_key, project_id, model_name, tag_name, output_dir, color, probability_threshold):
    tasks = []
    for image_file in sorted(os.listdir(image_dir), key=natural_sort_key):
        if image_file.endswith(".jpg"):
            image_path = os.path.join(image_dir, image_file)
            tasks.append(async_detect_objects(image_path, predictor_endpoint, api_key, project_id, model_name, tag_name, output_dir, color, probability_threshold))
    
    await asyncio.gather(*tasks)

# GIF 생성 함수
def create_gif_from_images(input_dir, output_gif_path, duration=100):
    image_files = sorted([os.path.join(input_dir, file) for file in os.listdir(input_dir) if file.endswith(".jpg")], key=natural_sort_key)
    if not image_files:
        print("No images found in the specified directory.")
        return

    frames = [Image.open(image_file).convert("RGB") for image_file in image_files]
    frames[0].save(
        output_gif_path,
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        duration=duration,
        loop=0
    )
    print(f"GIF saved at {output_gif_path}")

# 디렉토리 삭제 함수
def delete_directories(directories):
    for directory in directories:
        if os.path.exists(directory):
            shutil.rmtree(directory)
            print(f"Deleted directory: {directory}")

# 메인 비동기 함수
async def main():
    video_path = "D:/msaischool/videotest/test.mp4"
    output_dir = "D:/msaischool/videotest/output_frames"
    processed_dir = "D:/msaischool/videotest/processed_frames"
    output_gif_path = "D:/msaischool/videotest/output_animation.gif"

    # 디렉토리 생성
    for directory in [output_dir, processed_dir]:
        if not os.path.exists(directory):
            os.makedirs(directory)

    frame_interval = 2
    save_frames(video_path, output_dir, frame_interval)

    # 비동기 루프 실행
    await process_images_async(output_dir, prediction_endpoint1, prediction_key1, project_id_model1, model_name_model1, "Kick-scooter", processed_dir, "blue", 90)
    await process_images_async(processed_dir, prediction_endpoint2, prediction_key2, project_id_model2, model_name_model2, "unhelmet", processed_dir, "magenta", 90)

    # GIF 생성 (모든 프레임 포함)
    create_gif_from_images(processed_dir, output_gif_path, duration=100)

    # 디렉토리 삭제 (on/off 방식)
    delete_directories([output_dir, processed_dir])

if __name__ == "__main__":
    asyncio.run(main())


Saved frame: D:/msaischool/videotest/output_frames\frame_0.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_2.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_4.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_6.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_8.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_10.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_12.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_14.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_16.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_18.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_20.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_22.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_24.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_26.jpg
Saved frame: D:/msaischool/videotest/output_frames\frame_28.jpg
Saved frame: D:/msaischool/videotest/output_f